In [ ]:
pwd

In [8]:
import pandas as pd
import numpy as np
import glob
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import re

def jaccard_distance(x,y):
    num = sum(min(x,y))
    denom = sum(max(x,y))
    return abs(1.0*num/denom)

In [9]:
#data_path = '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_APG_bs/run_*_scan*_TwoSixReconstruction_0_1_600.csv'
data_path = '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_*_scan*_TwoSixReconstruction_0_1_600.csv'

In [10]:
file_list = glob.glob(data_path)

In [5]:
file_list

['/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_101_scan0021_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_135_scan0048_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_137_scan0021_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_140_scan0021_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_143_scan0021_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_146_scan0048_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_148_scan0021_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_bs/run_125_scan0048_TwoSixReconstruction_0_1_600.csv',
 '/export/jrobles/ARMY_Quick_look/segmentedFeatu

In [284]:
all_features = []
all_features.append('FOM')
all_features.append('FOMFrac')
# all_features.append('FracMu')
# all_features.append('FracSigma')
all_features.append('GeoMeanFractionalScattering')
# all_features.append('GeoMeanPThetaSquared')
# all_features.append('GeoMeanTheta')
#all_features.append('LogPTheta')1
# all_features.append('LogPThetaSquared')
# all_features.append('MeanDoCA')
# all_features.append('MeanFractionalScattering')
# all_features.append('MeanTheta')
# all_features.append('MikesFOM')
# all_features.append('Mu')
# all_features.append('NumStoppedTracks')
# all_features.append('NumTracks')
# all_features.append('PoCALogScat')
# all_features.append('PoCARate')
# all_features.append('PoCAScat')
# all_features.append('ScatteringRatio')
# all_features.append('Sigma')
all_features.append('SmoothedNumPoCAs')
# all_features.append('StoppedFracP')
# all_features.append('StoppedMean')
# all_features.append('StoppedMeanP')
all_features.append('StoppedThroughRatio')
all_features.append('box_mean')
all_features.append('box_p25')
all_features.append('box_p50')
all_features.append('box_p75')
all_features.append('box_p95')
all_features.append('box_std')

In [285]:
# loop over files and for each run and each scan contruct distance matrix and then cluster. 
final = pd.DataFrame()
for filename in file_list:
    path1 = filename #file_list[0]
    r = re.search(r'\d+',path1).group(0)
    df = pd.read_csv(path1)
    # pull number of scans and number of materials in the dataset
    scans = df.scan.unique()
    lengthofscan = len(df.tag.unique())
    materials = df.tag.unique()
    materialsclean = [re.sub(r'[0-9]+', '', x) for x in materials]

    # create distance matrix between materials
    for sc in range(0, len(df.scan.unique())):
        w, h = lengthofscan, lengthofscan;
        result = [[0 for x in range(w)] for y in range(h)] 
        rc=0
        cc=0
        r1 = (scans[sc]-1)*lengthofscan
        r2 = scans[sc]*lengthofscan
        for i in range(r1,r2):
            cc=0
            for z in range(r1,r2):
                x = list(df.loc[i,df.columns.isin(all_features)])
                y = list(df.loc[z,df.columns.isin(all_features)])
                result[cc][rc]=jaccard_distance(x,y)
                cc+=1
            rc+=1    
        #
        result_df = pd.DataFrame(result)
        result_df= abs(result_df)
        t1=pd.DataFrame(result_df.mean())
    #actual clustering 
        dbs=DBSCAN(eps=float(t1.quantile(0.8)), min_samples=1, leaf_size=5).fit((result_df))
        t=pd.Series(dbs.labels_,index=result_df.index)
    # create assignment dataframe and merge in the main one    
        m = pd.Series(materials)
        m1 = pd.Series(materialsclean)
        final1= pd.concat([t,m,m1], axis = 1)
        final1['run'] = r
        final1['scan']=sc
        final = final.append(final1)


In [286]:
#final[0:10]#.to_csv('ClusteredRunswithoutLogPThetaonsegmentedFeaturesConsolidated_bs.csv')
#path1 = '/export/jrobles/ARMY_Quick_look/segmentedFeaturesConsolidated_APG_bs/run_64_scan0086_TwoSixReconstruction_0_1_600.csv'final['Actual'] = final.index
final.columns = ['Predicted','Name', 'NameShort','run','scan','Actual']

In [294]:
final

,Predicted,Name,NameShort,run,scan,Actual
0,0,AmmNit7,AmmNit,101,0,0
1,1,Sugar5,Sugar,101,0,1
2,2,dummy,dummy,101,0,2
0,0,AmmNit7,AmmNit,101,1,0
1,1,Sugar5,Sugar,101,1,1
2,2,dummy,dummy,101,1,2
0,0,AmmNit7,AmmNit,101,2,0
1,0,Sugar5,Sugar,101,2,1
2,1,dummy,dummy,101,2,2
0,0,AmmNit7,AmmNit,101,3,0


In [287]:
## evaluate how good the clustering is
goodnessScoreF = pd.DataFrame()
# loop over all runs and all scans
runs = final.run.unique()
for run1 in runs:
#run1 = '101'
    scans = final.loc[(final['run']==run1),'scan' ].unique()
    for scan1 in scans:
    #scan1=0
### score goodness of scan
        goodnessScore = pd.DataFrame(columns=['run','scan','Perfect','Good','Bad'])
        goodnessScore['run'] = [run1]
        goodnessScore['scan'] = [scan1]
        predictedcats = (final.loc[(final['scan']==scan1) & (final['run']==run1),['Predicted']])
        actualcats = (final.loc[(final['scan']==scan1) & (final['run']==run1),['Actual']])
        predictedU = predictedcats.Predicted.unique()
        actualU = actualcats.Actual.unique()
        dfh = final.loc[(final['scan']==scan1) & (final['run']==run1)]
        ## clusters per material type
        m = dfh.groupby('NameShort')['Predicted'].apply(list)
        sets = [set() for i in range(len(m))]
        for i in range(0,len(m)):
            sets[i]= set(m[i])
        # all permutations of size 2 between the materials
        # checks if there is overlap - clusters with different materials in them
        z = list(itertools.combinations(sets, 2))
        u = z
        for i in range(len(z)):
            u[i] = set.intersection(*z[i])            
        ninters = sum([1 for x in u if len(x)>0])  

        if len(predictedU)==len(actualU) and ninters==0:
            goodnessScore['Perfect']=[1]
        if len(predictedU)<>len(actualU) and ninters==0:
            goodnessScore['Good']=[1]
        if ninters<>0:
            goodnessScore['Bad']=[1]
        goodnessScoreF = goodnessScoreF.append(goodnessScore)

#goodnessScoreF

In [288]:
goodnessScoreF.sum()

run                 inf
scan       5.487700e+04
Perfect    5.860000e+02
Good       2.100000e+01
Bad        1.721000e+03
dtype: float64

Code from Sean Starts here:

In [ ]:
for f in all_features:
    plt.figure()
    plt.title(f)5
#     print f,df[f].mean(),df[f].std()
    frange = (df[f].min(),df[f].max())
    for tv in truth_values:
        plt.hist(df[df[tv]==1][f],color=colors[tv],range=frange,bins=30,histtype='step')
#         plt.ylim(0,10)
    plt.legend([q[2:] for q in truth_values],bbox_to_anchor=[1.42,1])

In [ ]:
df[['scan','tag','FOM','GeoMeanFractionalScattering',
 'SmoothedNumPoCAs',
 'StoppedThroughRatio',
 'box_mean',
 'box_p25',
 'box_p50',
 'box_p75',
 'box_p95',
 'box_std']]

In [ ]:
for f1 in all_features:
    for f2 in all_features:
        plt.figure()
        plt.title(f2+' vs '+f1)
#         print f,df[f].mean(),df[f].std()
#         frange = (df[f].min(),df[f].max())
        for tv in truth_values:
            plt.scatter(df[df[tv]==1][f1],df[df[tv]==1][f2],color=colors[tv])
#             plt.ylim(0,10)
        plt.legend([q[2:] for q in truth_values],bbox_to_anchor=[1.42,1])

In [ ]:
df.columns

In [ ]:
len(file_list)